In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

In [7]:
cin = 3 # input channel size
cout = 4 # output channel size
k_size = 2 # kernel size
stride = 1
padding = 1
dilation = 2

# create_data
torch.manual_seed(42)
n = 2
lin = 4
x = torch.randn(n,cin,lin)

# convolution with module
conv = nn.Conv1d(cin, cout ,k_size, stride, padding, dilation=dilation)
linear = nn.Linear(5,3)
mout = conv(x)

# --------------- scratch  --------------
# padding
x_padded = F.pad(x, (padding, padding), 'constant', 0)
# convolution with scratch
lout = (lin + 2*padding - dilation*(k_size-1) - 1)//stride + 1
sout = torch.zeros(n, cout, lout)
for i in range(n):
    for j in range(cout):
        for l in range(lout):
            trg = x_padded[i, :, l:l + k_size + dilation - 1:dilation]
            sout[i,j,l] = torch.sum(trg*conv.weight[j]) + conv.bias[j]

print('------------------------------- with_scratch -------------------------\n', sout)
print()
print('------------------------------- with_module --------------------------\n', mout)
print()
print('---------------------------------- valid -----------------------------\n',
torch.round(mout, decimals=5)==torch.round(sout, decimals=5))

------------------------------- with_scratch -------------------------
 tensor([[[-0.3670,  0.5096, -0.4808,  0.1001],
         [ 0.4391,  0.5202,  0.4944,  0.3177],
         [ 0.6329,  0.1589, -1.3393, -0.0122],
         [-0.2328,  0.2560,  1.1390, -0.0174]],

        [[-0.0180,  0.5075, -0.2094, -0.1865],
         [ 0.0300, -0.4103, -0.0334, -0.1297],
         [ 0.3028, -0.0608,  0.0752,  1.0516],
         [-0.1494, -0.1147, -0.0413, -1.0659]]], grad_fn=<CopySlices>)

------------------------------- with_module --------------------------
 tensor([[[-0.3670,  0.5096, -0.4808,  0.1001],
         [ 0.4391,  0.5202,  0.4944,  0.3177],
         [ 0.6329,  0.1589, -1.3393, -0.0122],
         [-0.2328,  0.2560,  1.1390, -0.0174]],

        [[-0.0180,  0.5075, -0.2094, -0.1865],
         [ 0.0300, -0.4103, -0.0334, -0.1297],
         [ 0.3028, -0.0608,  0.0752,  1.0516],
         [-0.1494, -0.1147, -0.0413, -1.0659]]],
       grad_fn=<ConvolutionBackward0>)

---------------------------------

In [6]:
# ----------- backward ---------------
# create demo grad
grad = torch.randn(n, cout, lout)
# initialize grad
dw = torch.zeros(conv.weight.shape)
db = torch.zeros(conv.bias.shape)

# back propagation
# dw (kernel weight)
for k in range(cout): # kernel_ (size=cout)
    for i in range(cin): # w_column_ (size=cin)
        for j in range(k_size): # w_row_ (size=k_size) 
            g_trg = grad[:, k, :]
            x_trg = x_padded[:, i, j*dilation : j*dilation + cout]
            element = torch.sum(g_trg * x_trg)
            dw[k, i, j] = element
# db (bias)
db = torch.sum(grad, axis=(0,2))
print('-------- kernel gradient --------\n',dw)
print('---------- bias gradient---------\n', db)

-------- kernel gradient --------
 tensor([[[ 3.4104, -0.8469],
         [-1.4887, -2.3178],
         [ 0.3626,  1.9173]],

        [[-2.3558, -4.3232],
         [ 1.1275,  4.9704],
         [ 0.5073, -0.1646]],

        [[ 0.1340, -5.2385],
         [-0.5868,  2.6437],
         [-0.0690, -1.0572]],

        [[ 0.4253,  1.9225],
         [ 1.7639,  4.9373],
         [ 1.6168, -0.3467]]])
---------- bias gradient---------
 tensor([-0.2521, -3.1148,  2.1236, -4.1632])


In [9]:
import numpy as np

class Conv1d():
    """
    Note that this Class dose NOT use im2col algorithm.
    Therefore, it is not practical.
    """
    def __init__(self, cin: int, cout: int, k_size: int,
                    stride: int, padding: int, dilation: int):
        """
        cin : input_channel_size_
        cout : output_channel_size_
        k_size : kernel_size_
        """
        self.cin = cin
        self.cout = cout
        self.k_size = k_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        # initialize params
        self.weight = np.random.randn(cout, cin, k_size) # kernel weight
        self.bias = np.zeros(cout)

    def __call__(self, x: np.ndarray) -> np.ndarray: 
        """ forward_propagation
        x : input_data_ (batch_size, cin, lin)
        out : output_data_ (batch_size, cout, lout)
        """
        # padding
        x_padded = np.pad(x, [(0,0),(0,0),(self.padding, self.padding)], 'constant')
        self.x_padded = x_padded
        # convolution
        lout = (lin + 2*padding - self.dilation*(self.k_size-1) - 1)//stride + 1
        output = torch.zeros(n, cout, lout)
        for i in range(n):
            for j in range(cout):
                for l in range(lout):
                    trg = x_padded[i, :, l:l + self.k_size + self.dilation - 1:self.dilation]
                    output[i,j,l] = torch.sum(trg*conv.weight[j]) + conv.bias[j]
        return output

    def backward(self, grad: np.ndarray) -> np.ndarray:
        """ back_propagation
        grad : previous_gradient_ (batch_size, cout, lout)
        dx : gradient_ (batch_size, cin, )
        dw : kernel_gradient_ (cout, cin, k_size)
        db : bias_gradient_ (cout)
        """
        # dx
        
        # dw
        for k in range(self.cout): # kernel_num_ (size=cout)
            for i in range(self.cin): # w_column_ (size=cin)
                for j in range(self.k_size): # w_row_ (size=k_size) 
                    g_trg = grad[:, k, :]
                    x_trg = x_padded[:, i, j*self.dilation : j*self.dilation + cout]
                    element = np.sum(g_trg * x_trg)
                    dw[k, i, j] = element
        # db
        db = np.sum(grad, axis=(0,2))
        return 